In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Logistic/

/content/drive/MyDrive/Logistic


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read data

In [ ]:
daily_demand = pd.read_csv('data/daily_demand.csv')

- Kiếm tra kiểu dữ liệu từng cột

In [ ]:
# TODO: check dtypes & preprocess 
daily_demand['date'] = pd.to_datetime(daily_demand['date'])
daily_demand['product_id'] = daily_demand['product_id'].astype(str)
daily_demand.dtypes

product_id                      object
date                    datetime64[ns]
demand                           int64
Description                     object
Group                           object
Brand                           object
CasePerPallet                    int64
KgPerCase                      float64
ShelfLifeInDays                  int64
NetSalesValuePerCase             int64
COGSPerCase                      int64
ProfitPerCase                    int64
DailyProfit                      int64
ShelfLifeLabel                  object
StockMovement                   object
Profitability                   object
Perish                          object
dtype: object

# Explore

- Quan sát tên cột

In [ ]:
daily_demand.columns

Index(['product_id', 'date', 'demand', 'Description', 'Group', 'Brand',
       'CasePerPallet', 'KgPerCase', 'ShelfLifeInDays', 'NetSalesValuePerCase',
       'COGSPerCase', 'ProfitPerCase', 'DailyProfit', 'ShelfLifeLabel',
       'StockMovement', 'Profitability', 'Perish'],
      dtype='object')

- Kiểm tra tỷ lệ giá trị bị thiếu, giá trị nhỏ nhất/lớn nhất và số giá trị độc nhất của từng cột số

In [ ]:
nume_cols = daily_demand.select_dtypes([np.int64, np.float64, np.datetime64]).columns 
def missing_ratio(s):
    return s.isna().sum() * 100 / len(s)

daily_demand[nume_cols].agg([missing_ratio, pd.Series.min, pd.Series.max, pd.Series.nunique])

,date,demand,CasePerPallet,KgPerCase,ShelfLifeInDays,NetSalesValuePerCase,COGSPerCase,ProfitPerCase,DailyProfit
missing_ratio,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
min,2019-03-01 00:00:00,1.0,84.0,0.99,180.0,157000.0,47100.0,109900.0,210380.0
max,2022-03-31 00:00:00,671.0,270.0,3.89,730.0,807000.0,248150.0,581040.0,135311860.0
nunique,1127,404.0,9.0,15.00,3.0,21.0,22.0,22.0,3125.0


- Kiểm tra số giá trị bị thiếu và số giá trị độc nhất của từng cột chữ

In [ ]:
cat_cols = daily_demand.select_dtypes(include=["object"])
cat_cols.apply([missing_ratio, pd.Series.nunique])

,product_id,Description,Group,Brand,ShelfLifeLabel,StockMovement,Profitability,Perish
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nunique,23.0,23.0,7.0,8.0,3.0,3.0,3.0,9.0


# Feature engineering
Bước này tạo thêm cột mới

- Thêm cột `ProfitPerCase` và `DailyProfit`

In [ ]:
daily_demand['ProfitPerCase'] = daily_demand['NetSalesValuePerCase'] - daily_demand['COGSPerCase']
daily_demand['DailyProfit'] = daily_demand['ProfitPerCase'] * daily_demand['demand']

- Thêm cột `ShelfLifeLabel`

In [ ]:
# shelf-life label (redefine)
def shelf_life_label(days):
    if days <= 180:
        return 'Short'
    elif 181 <= days <= 370:
        return 'Medium'
    elif 400 <= days <= 750:
        return 'Long'

daily_demand['ShelfLifeLabel'] = daily_demand['ShelfLifeInDays'].apply(lambda x: shelf_life_label(x))
daily_demand['ShelfLifeLabel'].unique()        

array(['Long', 'Medium', 'Short'], dtype=object)

- Thêm cột `StockMovement` và `Profitability`

In [ ]:
daily_sales = daily_demand['demand'].sort_values(ascending=False)
n = len(daily_sales)
lst = [] # [[], [], []]
start = 0; end = 0

for pct in [0.7, 0.2, 0.1]:
    end += int(np.round(pct*n))
    lst.append(daily_sales.iloc[start:end + 1]) 
    start = end+1
    n -= (end-start+1)

daily_demand['StockMovement'] = np.nan
daily_demand['Profitability'] = np.nan

daily_demand.iloc[lst[0].index, daily_demand.columns.get_loc('StockMovement')] = 'Fast'
daily_demand.iloc[lst[0].index, daily_demand.columns.get_loc('Profitability')] = 'A'

daily_demand.iloc[lst[1].index, daily_demand.columns.get_loc('StockMovement')] = 'Normal'
daily_demand.iloc[lst[1].index, daily_demand.columns.get_loc('Profitability')] = 'B'

daily_demand.iloc[lst[2].index, daily_demand.columns.get_loc('StockMovement')] = 'Slow'
daily_demand.iloc[lst[2].index, daily_demand.columns.get_loc('Profitability')] = 'C'

print(daily_demand['StockMovement'].unique())
daily_demand['Profitability'].unique()

['Fast' 'Normal' 'Slow']


array(['A', 'B', 'C'], dtype=object)

- Thêm cột `Perish`

In [ ]:
conditions = [
    (daily_demand['ShelfLifeLabel'] == 'Long') & (daily_demand['StockMovement'] == 'Slow'),
    (daily_demand['ShelfLifeLabel'] == 'Long') & (daily_demand['StockMovement'] == 'Normal'),
    (daily_demand['ShelfLifeLabel'] == 'Long') & (daily_demand['StockMovement'] == 'Fast'),
    (daily_demand['ShelfLifeLabel'] == 'Medium') & (daily_demand['StockMovement'] == 'Slow'),
    (daily_demand['ShelfLifeLabel'] == 'Medium') & (daily_demand['StockMovement'] == 'Normal'),
    (daily_demand['ShelfLifeLabel'] == 'Medium') & (daily_demand['StockMovement'] == 'Fast'),
    (daily_demand['ShelfLifeLabel'] == 'Short') & (daily_demand['StockMovement'] == 'Slow'),
    (daily_demand['ShelfLifeLabel'] == 'Short') & (daily_demand['StockMovement'] == 'Normal'),
    (daily_demand['ShelfLifeLabel'] == 'Short') & (daily_demand['StockMovement'] == 'Fast')
]

values = ['C1', 'C2', 'C3', 'B1','B2','B3','A1','A2','A3']

daily_demand['Perish'] = np.select(conditions, values)
daily_demand['Perish'].unique()

array(['C3', 'C2', 'B3', 'B2', 'B1', 'A3', 'A2', 'A1', 'C1'], dtype=object)

- Thêm cột `Summer`: mùa hè là 1, mùa còn lại là 0

In [ ]:
daily_demand['Summer'] = daily_demand['date'].dt.month.isin([5,6,7]).astype(int)
daily_demand['Summer'].unique()

array([0, 1])

- Thêm cột `EventDay`: ngày lễ là 1, ngày thường là 0

In [ ]:
holiday_df = pd.read_csv('data/holidays.csv')
holiday_df['month'] = holiday_df['date'].apply(lambda x: x.split('/')[-1]).astype(int)
holiday_df['day'] = holiday_df['date'].apply(lambda x: x.split('/')[0]).astype(int)

def is_event(x):
    global holiday_df
    for m_event, d_event in zip(holiday_df['month'], holiday_df['day']):
        if x.month == m_event and x.day == d_event:
            return 1
    return 0

daily_demand['EventDay'] = daily_demand['date'].apply(is_event)
daily_demand['EventDay'].unique()

array([0, 1])

- Quan sát dataframe tổng thể sau khi thực hiện feature engineering

In [ ]:
daily_demand # final version

,product_id,date,demand,Description,Group,Brand,CasePerPallet,KgPerCase,ShelfLifeInDays,NetSalesValuePerCase,COGSPerCase,ProfitPerCase,DailyProfit,ShelfLifeLabel,StockMovement,Profitability,Perish,Summer,EventDay
0,12385000,2019-03-01,87,MILO Chocolate Malt Cup 18x55g TH,CUP,MILO,270,0.99,730,157000,47100,109900,9561300,Long,Fast,A,C3,0,0
1,12385000,2019-03-02,321,MILO Chocolate Malt Cup 18x55g TH,CUP,MILO,270,0.99,730,157000,47100,109900,35277900,Long,Fast,A,C3,0,0
2,12385000,2019-03-03,111,MILO Chocolate Malt Cup 18x55g TH,CUP,MILO,270,0.99,730,157000,47100,109900,12198900,Long,Fast,A,C3,0,0
3,12385000,2019-03-04,102,MILO Chocolate Malt Cup 18x55g TH,CUP,MILO,270,0.99,730,157000,47100,109900,11209800,Long,Fast,A,C3,0,0
4,12385000,2019-03-05,113,MILO Chocolate Malt Cup 18x55g TH,CUP,MILO,270,0.99,730,157000,47100,109900,12418700,Long,Fast,A,C3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25916,12435418,2022-03-27,73,OREO Mint Stick 18x60g TH,STICK,OREO,270,1.08,730,267000,88110,178890,13058970,Long,Fast,A,C3,0,0
25917,12435418,2022-03-28,40,OREO Mint Stick 18x60g TH,STICK,OREO,270,1.08,730,267000,88110,178890,7155600,Long,Fast,A,C3,0,0
25918,12435418,2022-03-29,46,OREO Mint Stick 18x60g TH,STICK,OREO,270,1.08,730,267000,88110,178890,8228940,Long,Fast,A,C3,0,0
25919,12435418,2022-03-30,82,OREO Mint Stick 18x60g TH,STICK,OREO,270,1.08,730,267000,88110,178890,14668980,Long,Fast,A,C3,0,0


- Lưu dataframe trên vào thư mục data/

In [ ]:
# save to .csv
daily_demand.to_csv('data/daily_demand.csv', index=False)